In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AR_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
605,5028007010,AR,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3039,2991708608,AR,"720 Ramsey St, Batesville, AR",35.763724,-91.645522,NaN
3091,8762207009,AR,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,AR,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145777,NaN
4672,8811267002,AR,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5177,4507858300,AR,"528 Central Ave Ste 100, Hot Springs National ...",34.511737,-93.054350,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('05')] #AR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/6 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

6


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14541/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
605,5028007010,AR,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,[]
3039,2991708608,AR,"720 Ramsey St, Batesville, AR",35.763724,-91.645522,[050634906002022]
3091,8762207009,AR,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,[]
4316,7127367007,AR,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145777,[]
4672,8811267002,AR,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,[]
5177,4507858300,AR,"528 Central Ave Ste 100, Hot Springs National ...",34.511737,-93.054350,[050510108002005]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14541/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
605,5028007010,AR,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3039,2991708608,AR,"720 Ramsey St, Batesville, AR",35.763724,-91.645522,050634906002022
3091,8762207009,AR,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,AR,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145777,NaN
4672,8811267002,AR,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5177,4507858300,AR,"528 Central Ave Ste 100, Hot Springs National ...",34.511737,-93.054350,050510108002005


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
605,5028007010,AR,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3091,8762207009,AR,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,AR,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145777,NaN
4672,8811267002,AR,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
605,5028007010,NaN
3039,2991708608,050634906002022
3091,8762207009,NaN
4316,7127367007,NaN
4672,8811267002,NaN
5177,4507858300,050510108002005


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,13574,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,13575,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,13576,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,13577,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,13578,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,13574,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13,NaN
1,13575,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13,NaN
2,13576,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13,NaN
3,13577,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13,NaN
4,13578,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13,NaN
...,...,...,...,...,...,...,...,...
5852,19426,7337827110,AR,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,5.125011e+13,NaN
5853,19427,7492638402,AR,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,5.125010e+13,NaN
5854,19428,7733137106,AR,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,5.119002e+13,NaN
5855,19429,9580078508,AR,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,5.143010e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,13574,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13,NaN,51430102011000.0
1,13575,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13,NaN,51399508001083.0
2,13576,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13,NaN,51190024031002.0
3,13577,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13,NaN,51430107053015.0
4,13578,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13,NaN,50434904001027.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5336097104,AR,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,51430102011000.0
1,5690507110,AR,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,51399508001083.0
2,2286469010,AR,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,51190024031002.0
3,8080007000,AR,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,51430107053015.0
4,5612127108,AR,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,50434904001027.0
...,...,...,...,...,...,...
5852,7337827110,AR,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,51250106013002.0
5853,7492638402,AR,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,51250101011005.0
5854,7733137106,AR,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,51190022091010.0
5855,9580078508,AR,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,51430101012019.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
605,5028007010,AR,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3091,8762207009,AR,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,AR,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145777,NaN
4672,8811267002,AR,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN


Convert to File

In [19]:
fips_AR = fips_merge_drop

In [20]:
fips_AR.to_csv("../../../data/state_data/geo/geo_fips/23/AR_fips_scraped.csv") 